# DSCI 100 Term Project
- name: Geoff Acabado
- student number: 59285189

## Familiarization
For this project, we will be looking how does different player characteristics correlate to thir probability of being subscribed to a game news paper.

In [ ]:
#Some libraries to install
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 10)
set.seed(9999)

### Tidying the data

In [ ]:
# install the data needed
player_data = read_csv("players.csv")
player_data



### Data explorations

In [ ]:
#Lets explore relationships between hours played, age and whether they are subscibed to game newspaper
player_plot_1 = player_data |>
    ggplot(aes(y = played_hours	, x = Age, color = subscribe)) +
    geom_point() +
    scale_y_log10() +
    labs(
        title = "Hours played Vs Age of Player",
        y = "Hours played by player (logarithmic scale)",
        x = "Age of player",
        colour = "Player is subscribed" )
player_plot_1

#### Figure 1.1 
From the figure above, it is easy to see that there are no obvious pattern between between hours played by the player and their age. Both variables also do not seem to be correlated with whether the players are subscribed or not. It is intersting to see however that everyone who played for 10 hours or more are all subscribed to a game newspaper. Another observation we see is that most players are arround 20 years old indicating that this is minecraft is popular among this demographic

### Data Analysis Plan

- Build a classification model that lets us categorize a player according to whether or not they are subscribed in a gaming newspaper
- Use their hours played in the game as well as their age as predictors
- Split the data into training and testing data
- Use a k-nn algorythm with a 5 fold cross validation to verify what the best K-value should be. 
- A K value that will give highest accuracy when used on testing data will be used to build the final model
- Use different metrics such as accuracy, precision, and recall to determine if data is useful in real world

### Model Tuning

In [ ]:
# Clean up data
player_data_clean = player_data |>
    select(subscribe, played_hours, Age) |>
    mutate(subscribe = as.factor(subscribe))

#split the data
player_data_split = initial_split(player_data_clean, prop = 0.7, strata = subscribe)
player_data_training = training(player_data_split)
player_data_testing = testing(player_data_split)

# Tuning the model with 5 fold cross validation
player_data_vfold = vfold_cv(player_data_training, v = 5, strata = subscribe)

#make recipe
player_data_recipe = recipe(subscribe ~., data = player_data_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

#Tune model
knn_spec = nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

#Get results
k_vals = tibble(neighbors = 1:25)
knn_results = workflow() |>
    add_recipe(player_data_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = player_data_vfold, grid = k_vals) |>
    collect_metrics() |>
    filter(.metric == "accuracy", !is.na(mean))

knn_results_plot = knn_results |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Nearest Neighbor" , y = "Mean Accuracy")+
    theme(text = element_text(size  = 12))
knn_results_plot

#### Figure 1.2
This graph shows mean accuracy for different values of K. We see that there is a local maximum at K = 5 and 6 which are probably the best K for this data set. We see that after K = 12, the accuracy increases again but that is probably just because there are a lot more people subscribed than not subscribed to news paper. As K increaases, the alogrythm will just classify any point as "TRUE" which results in high accuracy but is not very useful in this context.

### Final Model Building and Evaluation

In [ ]:
# Create final mode
final_spec = nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
  set_engine("kknn") |>
  set_mode("classification")

final_fit <- workflow() |>
    add_recipe(player_data_recipe) |>
    add_model(final_spec) |>
    fit(data = player_data_training)


#Evaluate final model
test_predictions <- predict(final_fit, player_data_testing) |>
  bind_cols(player_data_testing)

# Step 2: Calculate evaluation metrics
eval_metrics <- bind_rows(
  accuracy(test_predictions, truth = subscribe, estimate = .pred_class),
  precision(test_predictions, truth = subscribe, estimate = .pred_class),
  recall(test_predictions, truth = subscribe, estimate = .pred_class),
)

eval_metrics


#### Discussion


### Discussion and Improvements 

### Conclusions